## Audio experiments

In [235]:
#>flags -d:blas=cblas -p:~/code/misc/jnim

# Original Python code:
# srate = 11000
# dur = 1000
# rr = np.arange(dur*2)
# wave = np.sin(2*np.pi*400*rr/srate) * np.sin(0.5*np.pi*rr/dur)
# plt.plot(rr, wave)
# Audio(wave, rate=srate, autoplay=True)
    
import sequtils, std/[strutils, math, base64, strformat, times, os]
import arraymancer, pixie, jupyternimpkg/display
import wavfile, vorbis, fft

proc pixieHTML(img: Image): string =
    return fmt"""<img src="data:image/png;base64, {img.encodeImage(PngFormat).encode}" />"""

proc plot1D(img: Image, arr: openArray[float]): Image =
    var path: seq[string] = @[]
    
    let amax = arr[arr.maxIndex]
    let amin = arr[arr.minIndex]
    
    let mp = float(img.height)/(amax - amin)
    
    # For long arrays, draw <wmul> overlapping
    let wmul = 8
    if arr.len < img.width * wmul or false:
        for i in 0..<arr.len:
            let arr_v = arr[i]
            path.add(fmt"{int(float(i)*float(img.width)/float(arr.len))} {int((arr_v-amin) * mp)} ")
    else:
        for i in 0..<img.width * wmul:
            let arr_v = arr[int(float(i)*float(arr.len)/float(img.width)) div wmul]
            path.add(fmt"{i div wmul} {int((arr_v-amin) * mp)} ")
        
    img.strokePath(fmt"M {path[0]} L " & path.join(""), rgba(0, 0, 128, 255), strokeWidth = 1)
    return img

proc basicSine(srate: int): Tensor[system.float] =
    let srate = float(srate)
    let dur = 1000
    let rr_dur = dur * 2
    let rr = toSeq(0..<rr_dur).mapIt(it.float).toTensor()
    return sin(rr*(2.0*PI*440.0/srate)) *. sin(rr*(0.5*PI/float(dur)))

let wsample = loadWav("sample.wav")
#html wsample.toHTML(false)
#html pixieHTML(newImage(500,100).plot1D(wsample.toFloat))

let srate = 11025
let wave = basicSine(srate).toSeq
#html Audio(wave, srate, false)
#html pixieHTML(newImage(400, 100).plot1D(wave))

let vsample = loadVorbis("sample.ogg")
#html vsample.toHTML(false)

var vsf = vsample.toFloat.powerOf2Pad.mapIt(complex(it))
benchmark "fft":
    fft(vsf)
    
ifft(vsf)
let fftr = vsf.mapIt(it.re)[0..vsample.size div 2]
html Audio(fftr, vsample.freq, false)
html pixieHTML(newImage(500,100).plot1D(fftr))
    
#let fftr = vsf.mapIt(-abs(it))
#html pixieHTML(newImage(500,100).plot1D(fftr[0..fftr.len div 2]))

Your browser does not support the audio element.

<img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAfQAAABkCAYAAABwx8J9AAA5CElEQVR4Ae2dCbQtR1X3/31eQkCGMCMzyBCZIUAmktfnEUQhAobIDH4oigoulU8EFPMe70YgyCCi+An4yUJACSpGZkKSl34GEiAJwwcYwSDzJBBjIBPJ7W/9qs4+Xadud5/uM9zp1W+tXre7atfeu3ZVV/VQp+8OJRKJRCKR2PKkCT2RSCQSiW1AmtATiUQikdgGpAk9kUgkEoltQJrQE4lEIpHYBqQJPdGB3SdLeS4V+zsIJxKJRGIDSBN6ogNM5pAm9EQikdispAk90YE0oScSicRmJ03oiQ6kCT2xVUivhxIHLmlCT3QgTehbmwNpkkt9NXHgkib0RAfSILm56DtBH0jtdyDVNZGYJE3oiQ6kQXJz0bc9+spvZQ6kuiYSkxwgE3rfO5rEJGmQ3Fz0bY++8luZA6muicQkC5zQN/OkmU7y+Ujx21z0bQ/ksx1Scc7mqscyoK7A3/xnpGIfR4nEgcACJ3ROINiMv1VeT98284XNrBC/wYq0czV9XGYzQHtA13PNyR8iFWdTanvj6mrcWCrOsINEE4xZw0eki5+tzxIndDoJJ1fXQWeZ4Aeshy/raWu9oE7ZLqnct74T+nr2ofW0NS+0B3Ttz8hnN5CKD1Nqc7P7FCk/dvZ+Rl3H3FIq3js+SjRAzLIbbo3+kWhjiRM6nQS6DjrLpM0XN5DvkopiMR5gaz3uZvF7+PD1eYxKnTZiQscuLKwPqZk893nFfv93M9PXV+SzG0vFB6ta7T5Z2lVIO1e1qeo8fJykK2f3iboa2W2l4l12lGjCxeymUvGBJonE1mDH4tx0nULViRgfbyRtvpCXXU8qzl6Mh07fLqncV8ViGTg7PyEVZy5D+yTO1jrUKQa7sB6T7Hrampe+vjr5Q6Xi/ZVl0jaiTacx/EVJl81+7lCvMXeWitPGR4kGXMxuIRXvaRBIbBHShO4682DP4u5UODnWY6DEjrjr+tDy+xq21qNOMdiFNKFP0jcuyGc3qwZs7s4HK14n7bqZ7tLzp0n67mIm9OzuUn4Xf46vRx/aqtA/dBup+OetWoOEZ8kTOoPGZhgsXIcNnh6EkMeg1uWOmoEQ+bbBgfyu+uYBO2KQft88WrqBrfWoUwx2oS3ei2I9bc3LNF93nyLlx1aTopO/pVT8i7fMMe1pdOn760X+LElfq3zvC3Ubc29J/+GP1qMPbVWIWXYHqXjn5q5Bl/H3wGbGCT0eMAg0kzeDhE3grpMs4DEtutE1zwlJeajTQV6bn6F9ZKFOj4FMrC/UsSicnWCQXibO1gLasi/YhbZ4L4r1tDUv03wdnijpimpSdPLcgY3eJ3NMexqbaUIf/qakSyrf+0LdxjxQ0uf90Xr0oa0KMcvuIhV/t7lrgJ+Q2rKJGSf0/LGTC1cItA0QNjhYmh3PCnpg1kZkMuVio8kP9ON7Wz5gH1lgvwlkYn2kQVu5vjidP7k+i36wFdfJIL7kL7JuBnphGbpj1tPWvEzzNX+ipEurSRF5t0DsH7xld7zL70NT398I8t/xk/CsbU7dxjxE0qf90az6DgSIWXZ3qfjbja9t23iCn7DotmyzubWYcUIfniTpf6IBYzRA2OBA8EmzY5glcOiBWRuR8rEfIV3yAfvIAvtNIBPbIw2s3CxxiEFndnv/mOyFh0ofuTqWWBzOVsMdOnlgdVski9DdNdaLsGV0tTkr03wdPkPSdybPz8FvVetEKE97GuE5utHkvy/pU5XvIV3iSt2MwUly9dxM9duMuJjdSyr+ZuO9c760vB6FRY81bTZDuvS/jWXWCf0pkr5XnXQExAYIO3kszY73rEiDl/Rf/Y0emLURh4+VsmOa7aIf3/GT/finPKQB9sN9g0YePr76SRAy6GMLXz/wlIBj8tnHHjpnBT08JsvvJl3vQ9VgvQycrS06oeM78Sb2bfFGDtpkurJIXXU06acv0n+ze0/2d+TD9rNjY96+uEiGfyTp434hG36G7cExhGkxJgPUGTZT/TYb9BnODzeev2HjvbP2q2vjtrx56Kq3q9zGMeOEnv+SpG/2m9CHJ7RPrEDnImhhY3IMYVofWGST3WdygAtBPyc+Jz3YvtkjHzgO9w2X9oDq/RPH6AB0cuxOGHkfgHzy0Dkr6NXdJH1l+Xch2GrymTyYpy5NoJvYTZuM20AHvrM16aHfYWfeNjGwCcuICTTpJ516QlgXSyeNfeoaQnpfX4kZuvqWm8Zwr6RzK6lQP/YgTIsxGaiLRWIS4kWcSn4q+PrJvI0Af6CujdvyYuifwydO/lSzia56u8ptHLNO6L8m6cvdJnQ6CwNpdnMpe6if1OoaC+oCVpfWh+HvStmdmu2iHx8Z1MD2zUfygeNw33BpQ6n4C5/CMTog1AnhMfvonBVn556SvrRxEzonDZPDvHVpwtWx4clAV0wHNPlpMk35fUEfzNO+bTTpJ514QVgXSycNTMYgva+v6IS+5drYc5CUrfhz1Qj1d7FpMmD1nKV+mxHON+oXxqSO3S+V8odV43Mb6CNO5Q+l4s/aJNcH/IG6OpLHeMN8UpcfgqyOlIq3hKn1ONmGx/whXeU2jlkn9OdK+kLVYagonQLs5InTdGspe6A/WZsaw5U5ZPIjL6RBU5lp5Huk7NBmu+jHd/wG2zd75APH4b5BWvY4afhyqVj1jwrRAfxlM5psQNeT1XB27+PbARvoxsdl4Gzt8hcO4clk6bHtpro0pTfRpL8PpgNiPw2Tacrvy/ARkpb4BTb8rRvYSKedIKyLpZMGJmOQ3rfvoBP6lmtjeBMpe5E/V41QfxebJgNWz1nqtxmxuoUxqSNnjRMTdIevHqLTxelqqXhVnbb1BX+gznfy8LVLeyKrE7rVyckucULvO+7NzqwTOitRP9dzQr+9Wh99A4HlTj78BCFpDF45H1E5A6nuPGGHdKtXSNnADxLTOgnEHQb7QNlw3yBtwAD+Vqn4weSEHoNuNog7JXoAO21Y54DB0/0kC7G+RYJv5jd/bVK39PJjUnFWZZF02mz4ksl3+6TDtDoayGNvnrqZDmjSYzJN+X0ZPkrSNdX5sWia/LV0COti6aQBMQ0hvWubGOiE/Gek4c8u5ouFx95K2vE8f64aoV9mM0yLMRmwelI/0tnaym528B/4yxbXZTw23KFa42RpsayBHhen66TiVEtdLm2LePEH6vwlD19pz7r8EGQHPy8d/ZfSuVeEOZMQH8aqrjphmu0YfIG+5foz44Q+fKGkT1YDFg4TaLDAhGnkubvJ0XvkpopRRreT8vv5iRE50ihf8oh/9DvarjzhttKAVbMju+hiQ6/BsdP/r/6uyu0Hn20lHygT7hukUWb1/dL+S7zfHE8jtAHoAey0YXIQ2on1LRJs1tmy9PJi/xUyTg7SwORDvyxvWh0N5NET6uiL6YAmPWOZjy9mYnI/6/xRdX4sGvM3ro+lQ5hn6aQBMQ3heCcfFunx6U90MhBmO6XyC1L+AN/3u7ZtHfntpcFzZ5/Q6X/4FNfTjmEe/zYaYh4S12Wcf3dJX5dbEDx4sY9nKEuckCWNv7S/Mqn440o7MuFi30Vy/EelnO8i1DxBwB9oysNX2rMuPwQ9yGbvk/Z/JcyZxORindSfPLPDsfUtS+sKeqBvuf7MOqGzEpW7slGD4DDBAwtMmBZi+XVQRneR9C2fi37SnG5W1Xd4HxKy897S4Fd9CnYN9II1EmR5/d0u9oEy4b5BmvOPyWC0OtcdTwF/0GmgB8K0GPOXshDaIa2t7DzgW52tcfpX/DezOTZMPvTL8kM/qRPpYZpBOnpCHX0xHdCkx2SYmIp3Izkf7medLDIanR+LZuxvcOEJlg5hnqWTBsQ0puSR4xsnU7u0DZTfkMSGjjnqzC82OF/NTwj1UQ8I00LIp25Wnn3gLxvplI0/jLVVoH5MKmFdQsh33Hf0hby7Stn9/IR

CPU Time [fft] 0.500s

